# Outline : 
Understand the business ask, the result of this project should highlight:

    1. the value driven by iSurgery 
    2. opportunities to increase adoption across hospital locations

for a customer who had recently go-live with iSurgery 

# Background :
    iSurgery is a software product which helps free up capacity in operations rooms and create a more transparent and surgeon-centric provess for measuring Operating Room utilization.
 
# Procedure:
    1. Import data
    2. EDA 
    3. Assumptions 
    4. Metrics definition 
    5. Data preprocessing 
    6. Metrics calculation (for bullet 1)
    7. Cohort analysis (for bullet 2)
    8. Result and summary
    9. Improvement and next steps
    10. Limitations


## Importing data and EDA 

Libraries and settings

In [7]:
import sqlite3
import pandas as pd

# for exporting .db to csv, then to feed into Tableau for visualizations
conn = sqlite3.connect("TestDB.db")
# Only run once
# db_df = pd.read_sql_query("SELECT * FROM exchange_transactions", conn)
# db_df.to_csv('exchange_transactions.csv', index=False)

In [2]:
%load_ext sql
%sql sqlite:///TestDB.db

EDA

In [122]:
%%sql 

select min(created_datetime), max(created_datetime), min(snapshot_date), max(snapshot_date) 
FROM exchange_transactions

 * sqlite:///LeanTaaSTestDB.db
Done.


min(created_datetime),max(created_datetime),min(snapshot_date),max(snapshot_date)
2019-01-14 08:32:01.000000,2019-08-27 11:33:52.000000,2019-06-03 00:00:00.000000,2019-08-30 00:00:00.000000


In [146]:
%%sql 

SELECT strftime('%m', created_datetime) AS month, 
        count(distinct transaction_id) AS total_transactions 
FROM exchange_transactions 
GROUP BY 1 

 * sqlite:///LeanTaaSTestDB.db
Done.


month,total_transactions
01,62
02,151
03,212
04,589
05,1438
06,1966
07,1757
08,760


In [260]:
# %%sql 

# SELECT strftime('%m', created_datetime) AS month, action,
#         count(distinct transaction_id) AS total_transactions 
# FROM exchange_transactions 
# GROUP BY 1,2 

We have a Tableau viz for the query above: (EDA) Action by month

#### Narration of the chart:
 
- The first deny_release didn't happen until April.
- The first deny_transfer didn't happen until May.
- Prior to May, there were always more releases than requests. Ever since we got to May, there were more requests than releases.

#### Observations:
- Transaction creation times range from Jan 14,2019 to Aug 27,2019. 
- The associated blocks from the transactions range from Jun 3,2019 to Aug 30,2019. 
- The two ranges above tell us that there are no outliers in these two date columns.
- More and more transactions were made over time. June has the highest number of transactions, followed by July and then May.
- We see the least number of transaction in January.

In [ ]:
%%sql

SELECT location, 
        count(distinct room_name) AS number_of_rooms 
FROM exchange_transactions 
GROUP BY 1 
ORDER BY 2 DESC 

 * sqlite:///LeanTaaSTestDB.db
Done.


location,number_of_rooms
CENTER,35
MRAH OR,18
MRH OR,15
MSC OR,9
MASC OR,8
VISION OR,6
CENTER PEDS,4


In [ ]:
%%sql 
SELECT surgeon, 
        count(distinct location) 
FROM    
    (SELECT surgeon, location , count(1) as count 
        FROM exchange_transactions GROUP BY 1, 2 )    
GROUP BY 1 
HAVING count(distinct location) > 1 
ORDER BY count(distinct location) DESC
LIMIT 1 


 * sqlite:///LeanTaaSTestDB.db
Done.


surgeon,count(distinct location)
Daniel Finney,4


#### Observations:
- There are a total of 7 different locations and each location has different number of rooms. 
- Center has the most number of OR in total, and Center Peds , which should represent Pediatric in Center, only has four ORs. 
- Vision is the second lowest in OR rooms, with a total of 6.  
- A surgeon can travel across location in scheduling blocks, for example, surgeon Daniel Finney had blocks scheduled in four different locations.


In [ ]:
%%sql
SELECT distinct action 
FROM exchange_transactions 
WHERE room_name IS null 

 * sqlite:///LeanTaaSTestDB.db
Done.


action
REQUEST
DENY_REQUEST
MARK_UPDATED


In [135]:
%%sql 
SELECT COUNT(*)
FROM exchange_transactions where action= 'REQUEST' 
AND room_name IS NOT null 

 * sqlite:///LeanTaaSTestDB.db
Done.


COUNT(*)
24


#### Observations:
- Room_name is null only when the action is either request, deny_request, or mark_updated.
- However, there are also cases when action is request but the room name is not null. In those situations, scheduler specifies which room they are requesting.
- MARK_UPDATED only takes place when a request got approved. Therefore, we can use it as a measure of successful transaction.

# Assumptions
    
    1. All transaction made (from Jan to Aug) targetted blocks that take place in June, July and August. This is an interesting phenomeon and due to limited information, we will make the assumption that this is normal behavior.
    2. As a follow up for the first bullet, we will make the assumptions that schedulers were only instructed to make changes for blocks in June, July, and August.  
    3. Although the first transaction is created on 2019-01-14, we will make the assumption that the "recent go-live" started officially on 2019-01-01 for the sake of uniform number of days for each month. 
    4. Similarly, the last transaction made from our table is 2019-08-27, but we will make the assumption that we didn't generate this dataset until 2019-08-30, again for the sake of keeping consistent number of days for each month. 
    5. All hospital rooms that exist has been released, requested, or transfered at some point from 2019-01-14 to 2019-08-27, which is the period captuerd by exchange_transactions. In other words, count(distinct room_name) will get us the total number of rooms in this hospital for this customer.  
    6. All actions made by the schedulers were correct.



# Metrics definitions
Utilization:
- volume - number of release, request, transfer submitted

Adoption: 
- number of approved request, approved release and approved transfers
- percentage of approved (or denied) request - approve_request divided by total request 
- percentage of approved (or denied) transfer

Optimization: 
- how many hours of block were freed up? in total by month or on average  
- how many hours of block were optimized? 
- how many rooms were freed up? optimized?
- what is the percent of utilization of those freed up capacity? 
- which location has the highest adoption? highest optimization?

# Data preprocessing
In this section we will check for nulls and noisy data.

In [131]:
%%sql
SELECT count(1), count(distinct transaction_id) 
FROM exchange_transactions

 * sqlite:///LeanTaaSTestDB.db
Done.


count(1),count(distinct transaction_id)
6965,6935


In [154]:
%%sql
select count(distinct transaction_id) 
FROM (select transaction_id FROM exchange_transactions group by 1 having count() >1 )

 * sqlite:///LeanTaaSTestDB.db
Done.


count(distinct transaction_id)
20


In [159]:
%%sql
select * 
FROM exchange_transactions where transaction_id is null 

 * sqlite:///LeanTaaSTestDB.db
Done.


transaction_id,parent_transaction_id,action,scheduler,surgeon,created_datetime,snapshot_date,start_time,end_time,room_name,location


In [105]:
%%sql
SELECT * 
FROM exchange_transactions 
WHERE scheduler IS NULL 
OR surgeon IS NULL 
OR scheduler = surgeon 

 * sqlite:///LeanTaaSTestDB.db
Done.


transaction_id,parent_transaction_id,action,scheduler,surgeon,created_datetime,snapshot_date,start_time,end_time,room_name,location


In [186]:
%%sql
select distinct location 
FROM exchange_transactions where action = 'DENY_TRANSFER'

 * sqlite:///LeanTaaSTestDB.db
Done.


location
VISION OR
CENTER
MSC OR


In [218]:
%%sql
select * 
FROM exchange_transactions where end_time < start_time

 * sqlite:///LeanTaaSTestDB.db
Done.


transaction_id,parent_transaction_id,action,scheduler,surgeon,created_datetime,snapshot_date,start_time,end_time,room_name,location


#### Observations:
- Although it was said in the data dictionary that transaction_id is a uniquely generated serial number, the above queries show that there are 20 duplicated transaction_id's in the data table
- The usual approach I'll take is to handle this is to remove the duplicated rows using row_number() window function and filtering by row number if I were in PostgreSQL. However, for thhis assignemnt, I am using SQLite and when window funcion does not work the same way in this database, I'll take a different approach to use count distinct whenever I work with the transaction_id column

- We do not see anywhere in the data where scheduler name or the surgeon name is empty, which is good.  
- We do not see any rows where scheduler name is the same as surgeon name. 
- we do not see anywhere in the table where end time is less than start time, this is another good sign.
- From the above, we can infer that the data ingestion for these two columns is accurate.



# Metrics calculation - finding value with iSurgery
In order to uncover the value the customer has seen with iSurgery so far, we will use the optimization metrics defined in the previous section.

#### How many hours were freed up? How many hours were optimized? 

In [155]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration,
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
)
SELECT strftime('%m', snapshot_date) as month, ROUND(sum(block_duration),0) AS released_hours
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1 

 * sqlite:///LeanTaaSTestDB.db
Done.


month,released_hours
06,2951.0
07,3481.0
08,3034.0


In [138]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
)
SELECT strftime('%m', snapshot_date) as month, ROUND( sum(block_duration),0) AS optimized_hours
FROM cte 
WHERE ( action = 'REQUEST' AND or_action = 'APPROVE_REQUEST' ) 
OR ( action  ='TRANSFER' AND or_action = 'APPROVE_TRANSFER')
GROUP BY 1 

 * sqlite:///LeanTaaSTestDB.db
Done.


month,optimized_hours
06,2034.0
07,2532.0
08,2472.0


In [5]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
), 
cte_a as ( 
SELECT strftime('%m', snapshot_date) as month, ROUND(sum(block_duration),0) AS optimized_hours
FROM cte 
WHERE ( action = 'REQUEST' AND or_action = 'APPROVE_REQUEST' ) 
OR (action  ='TRANSFER' AND or_action = 'APPROVE_TRANSFER' )
GROUP BY 1 
), 
cte_b as( 
SELECT strftime('%m', snapshot_date) as month, ROUND(sum(block_duration),0) AS released_hours
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1 
)
SELECT cte_a.month as 'Month', 
    released_hours as 'Freed hrs', 
    optimized_hours as 'Opt hours', 
    round(optimized_hours /released_hours,2) as '% utilized' 
FROM cte_a join cte_b on cte_a.month = cte_b.month 


 * sqlite:///LeanTaaSTestDB.db
Done.


Month,Opt hours,Freed hrs,% utilized
06,2034.0,2951.0,0.69
07,2532.0,3481.0,0.73
08,2472.0,3034.0,0.81


We have a Tableau visualization which shows the breakdown of released block hours by location. (Released block hrs by location)

#### Narration of the chart:
- CENTER has the most released hours at all times.
- MRAH is the second hightest in number of hours released, with an average between 1000-1200 from June to Aug.
- CENTER PEDS has the least number of block hours released.

Another Tableau viz was built in attempting to show the breakdown of optimized and released hour for the three months. 
However, a pie chart is not the best in showing inflow and outflow, it's sufficient to show how the slices of transfer + requests roughly equal to the slice of release.

This final percentage ( optimizaed/released percentage) represents the %utilized from the freed up capacity.

#### How many rooms were freed up? optimized? 

In [93]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
)
SELECT strftime('%m', snapshot_date) as block_month, count( distinct room_name)
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1 

 * sqlite:///LeanTaaSTestDB.db
Done.


block_month,count( distinct room_name)
06,87
07,90
08,85


In [230]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
),
cte_a as( 
SELECT location, strftime('%m', snapshot_date) as block_month, round(count( distinct room_name),0) as count
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1,2
),
cte_b as(
SELECT location, strftime('%m', snapshot_date) as block_month, round(count(distinct room_name),0) as countss
FROM exchange_transactions 
GROUP BY 1,2
)

SELECT cte_a.location, cte_a.block_month, round(count/countss,2) as percent_released
FROM cte_a JOIN cte_b 
ON cte_a.location = cte_b.location and cte_a.block_month = cte_b.block_month

 * sqlite:///LeanTaaSTestDB.db
Done.


location,block_month,percent_released
CENTER,06,0.94
CENTER,07,0.94
CENTER,08,0.94
CENTER PEDS,06,0.75
CENTER PEDS,07,0.75
CENTER PEDS,08,1.0
MASC OR,06,0.88
MASC OR,07,1.0
MASC OR,08,1.0
MRAH OR,06,1.0


In [267]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
)
SELECT strftime('%m', created_datetime) as month, 
    strftime('%m', snapshot_date) as block_month, 
    count( distinct room_name)
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1,2 
LIMIT 5 

 * sqlite:///LeanTaaSTestDB.db
Done.


month,block_month,count( distinct room_name)
01,06,10
01,07,5
02,06,12
02,07,15
02,08,2


We have a Tableau visualization for the table above ( Number of rooms released by Month of Creation )
##### Narration of the chart:

- The number of submission increases month over month, this can be seen as increasing adoption of the new tool.
- No rooms for blocks in August were released in Jan. 
- In March, we can see a >100% jump for the number of transaction submitted to release capacity in June and in August.
- As we get closer and closer to the month of the associated block, the number of transactions increases. 

In [171]:
%%sql 
SELECT strftime('%m', snapshot_date) as block_month, 
    count(distinct room_name) 
FROM exchange_transactions 
WHERE action = 'APPROVE_REQUEST' OR action ='APPROVE_TRANSFER' 
GROUP BY 1


 * sqlite:///LeanTaaSTestDB.db
Done.


block_month,count(distinct room_name)
06,80
07,89
08,83


In [6]:
%%sql 
with cte as (
SELECT at.*, ROUND((JULIANDAY(at.end_time)-JULIANDAY(at.start_time))*24,1) AS block_duration, 
et.scheduler AS or_scheduler, 
et.action AS or_action, 
et.created_datetime 
FROM exchange_transactions at 
JOIN exchange_transactions et 
ON at.transaction_id = et.parent_transaction_id 
), cte_a as( 
SELECT strftime('%m', snapshot_date) as block_month, round(count( distinct room_name),0) as released_rooms
FROM cte 
WHERE action = 'RELEASE' AND or_action = 'MARK_UPDATED' 
GROUP BY 1 
),
cte_b as(
SELECT strftime('%m', snapshot_date) as block_month, 
    round(count(distinct room_name),0) as opt_rooms
FROM exchange_transactions 
WHERE action = 'APPROVE_REQUEST' OR action ='APPROVE_TRANSFER' 
GROUP BY 1
) 

SELECT cte_b.block_month, released_rooms as 'Freed rooms',opt_rooms as 'Opt rooms', 
    round(opt_rooms/released_rooms,2) as '% utilized'
FROM cte_b join cte_a on cte_a.block_month = cte_b.block_month 



 * sqlite:///LeanTaaSTestDB.db
Done.


block_month,Freed rooms,Opt rooms,% utilized
06,87.0,80.0,0.92
07,90.0,89.0,0.99
08,85.0,83.0,0.98


#### Similar to total hours, we can measure %Utilization from the freed up room capacity.

#### Which location has the biggest adoption? 

Let's start with the total number of approved release, request and transfer.

In [197]:
%%sql
WITH cte AS (
 SELECT at.*, et.scheduler AS or_scheduler, et.action AS or_action, et.created_datetime 
 FROM exchange_transactions at JOIN exchange_transactions et ON at.transaction_id = et.parent_transaction_id 
 )
 SELECT COUNT(distinct transaction_id) FROM cte where action = 'RELEASE' and or_action = 'MARK_UPDATED' 

 * sqlite:///LeanTaaSTestDB.db
Done.


COUNT(distinct transaction_id)
1177


In [198]:
%%sql
WITH cte AS (
 SELECT at.*, et.scheduler AS or_scheduler, et.action AS or_action, et.created_datetime 
 FROM exchange_transactions at JOIN exchange_transactions et ON at.transaction_id = et.parent_transaction_id 
 )
 SELECT COUNT(distinct transaction_id) FROM cte where action = 'TRANSFER' and or_action = 'APPROVE_TRANSFER' 

 * sqlite:///LeanTaaSTestDB.db
Done.


COUNT(distinct transaction_id)
256


In [199]:
%%sql
WITH cte AS (
 SELECT at.*, et.scheduler AS or_scheduler, et.action AS or_action, et.created_datetime 
 FROM exchange_transactions at JOIN exchange_transactions et ON at.transaction_id = et.parent_transaction_id 
 )

SELECT COUNT(distinct transaction_id) 
FROM cte where action = 'REQUEST' AND or_action = 'APPROVE_REQUEST' 

 * sqlite:///LeanTaaSTestDB.db
Done.


COUNT(distinct transaction_id)
1134


Over the span of three months, there were 1177 approved releases, 1134 approved request and 256 approved transfer.

We have a Tableau viz to show the breakdown of approved releases, request and tranfer by snapshot month. (Transaction by Action Type) 

#### Narration of the chart:

- From June to Aug, number of approve_request were straightly increasing.
- The number of releases and approve_transfer spiked in July, but then dropped in August.

In [3]:
%%sql 
SELECT location, (sum( CASE WHEN action = 'APPROVE_REQUEST' THEN 1 ELSE 0 end )*100 /sum( CASE WHEN action = 'REQUEST' then 1 ELSE 0 end)* 100)/ 100 AS percentage 
FROM exchange_transactions
GROUP BY 1 
ORDER BY 2 DESC

 * sqlite:///LeanTaaSTestDB.db
Done.


location,percentage
CENTER,90
MRH OR,87
MASC OR,85
MRAH OR,84
VISION OR,83
MSC OR,82
CENTER PEDS,77


In [4]:
%%sql 
SELECT location, (sum( CASE WHEN action = 'APPROVE_TRANSFER' THEN 1 ELSE 0 end )*100 /sum( CASE WHEN action = 'TRANSFER' then 1 ELSE 0 end)* 100)/ 100 AS percentage 
FROM exchange_transactions
GROUP BY 1 
ORDER BY 2 DESC

 * sqlite:///LeanTaaSTestDB.db
Done.


location,percentage
MRH OR,100
MRAH OR,100
MASC OR,100
CENTER PEDS,100
MSC OR,98
CENTER,98
VISION OR,90


In [145]:
%%sql 
SELECT strftime('%m', created_datetime) as month, action, count(distinct transaction_id ) 
FROM exchange_transactions 
WHERE action = 'APPROVE_REQUEST' OR action = 'APPROVE_TRANSFER' 
GROUP BY 1,2

 * sqlite:///LeanTaaSTestDB.db
Done.


month,action,count(distinct transaction_id )
01,APPROVE_REQUEST,2
01,APPROVE_TRANSFER,4
02,APPROVE_REQUEST,4
02,APPROVE_TRANSFER,14
03,APPROVE_REQUEST,18
03,APPROVE_TRANSFER,10
04,APPROVE_REQUEST,63
04,APPROVE_TRANSFER,21
05,APPROVE_REQUEST,243
05,APPROVE_TRANSFER,51


Tableau visualization Adoption by Location

##### Narration of the chart: 

- MRAH OR has 468 request approvals, the highest of all locations. The second is CENTER, with 277 approvals.
- CENTER has the highest number of approved transfer, with a total of 125 cases. The nest is MSC OR, with 65 successful cases. 
- There weren't any approved transfer in MASC until August.
- July is the month with most activities in MRAH and MASC.
- June is the month with most activities in CENTER, MRH and MSC.

# Cohort analysis - opportunities to increase adoption

- Which location has the lowest adoption, i.e. highest denial rate? 
- What kind of block (durationwise) has the least approval? (in general and location specific)

#### Caution when using denial rate:
A denial rate can result from one of the two scenarios below: 
1. OR scheduler reviewed the request and determined the change is not doable
2. The original scheduler correcting a request they've created earlier by denying/cancelling the initial request. 

Case one is how we normally perceive a denial. Case two is not as common for there are only 89 transacions of such. Though not commone, it is important to raise awareness of case two for transparency. For the rest of the exercise, let's interpret all denial as explained in case one.  


In [217]:
# To find out about the 89 transactions of schedulers cancelling their initial request

# %%sql
# WITH cte AS (
# SELECT at.*, et.scheduler AS or_scheduler, et.action AS or_action, et.created_datetime 
# FROM exchange_transactions at JOIN exchange_transactions et ON at.transaction_id = et.parent_transaction_id 
# )
# SELECT * 
# FROM cte 
# WHERE scheduler = or_scheduler AND (action ='REQUEST' OR action = 'TRANSFER') 
# AND (or_action = 'DENY_TRANSFER' OR or_action = 'DENY_REQUEST')

#### Which location has the highest request denial rate? 

In [179]:
%%sql 
SELECT location, (sum( CASE WHEN action = 'DENY_REQUEST' THEN 1 ELSE 0 end )*100 /sum( CASE WHEN action = 'REQUEST' then 1 ELSE 0 end)* 100)/ 100 AS percentage 
FROM exchange_transactions
GROUP BY 1 
ORDER BY 2 DESC

 * sqlite:///LeanTaaSTestDB.db
Done.


location,percentage
CENTER PEDS,22
MSC OR,17
VISION OR,15
MRAH OR,15
MASC OR,14
MRH OR,12
CENTER,6


We have a viz for the query above. (Denial rate)

#### Narration of the chart:

- Request denial rate was straightly increasing for MRH.
- Request denial rate temporarily increased in Aug for MSC.
- Central PEDS has the highest request denial rate in June and July.
- Request denial rates were decreaasing for all locations except for MRH and MSC.

#### Which location has the highest transfer denial rate? 

In [192]:
%%sql 
SELECT COUNT( distinct location) 
FROM exchange_transactions where action ='TRANSFER'

 * sqlite:///LeanTaaSTestDB.db
Done.


COUNT( distinct location)
7


In [178]:
%%sql 
SELECT location, (sum( CASE WHEN action = 'DENY_TRANSFER' THEN 1 ELSE 0 end )*100 /sum( CASE WHEN action = 'TRANSFER' then 1 ELSE 0 end )* 100) / 100 AS percentage 
FROM exchange_transactions
GROUP BY 1 

 * sqlite:///LeanTaaSTestDB.db
Done.


location,percentage
CENTER,1
CENTER PEDS,0
MASC OR,0
MRAH OR,0
MRH OR,0
MSC OR,1
VISION OR,10


Only three out of seven locations encountered transfer denials, they are VISION, CENTER, and MSC.

#### What kind of blocks (durationwise) has the least approval? ( in general and location specific )

A new variable is created to measure duration: block_cat
- block_cat is calculated by grouping the duration of each associated block, where duration is computed by datediff('hour', end time, start time)

In [214]:
%%sql 

with cte as ( 
    SELECT *, 
        round((JULIANDAY(end_time)-JULIANDAY(start_time))*24,1) as block_duration
    FROM exchange_transactions
),
cte_a as (
    SELECT *, 
    case 
        when block_duration >=1 and block_duration <2 then '1-2 hours' 
        when block_duration >=2 and block_duration <3  then '2-3 hours' 
        when block_duration >=3 and block_duration <6 then '3-5 hours' 
        when block_duration >=6 and block_duration <=8 then '6-8 hours' 
        when block_duration >=9 and block_duration <10 then '9 hours'
        when block_duration >=10 and block_duration <11 then '10 hours'
        when block_duration >=11 and block_duration <12 then '11 hours'
        when block_duration >=12 then '12 hours'
        else 0
    end as block_cat 
    FROM cte
),
cte_b as(
    select at.*, cte_a.block_duration, cte_a.block_cat, cte_a.scheduler as or_scheduler, cte_a.action as or_action, cte_a.created_datetime 
    from exchange_transactions at join cte_a on at.transaction_id = cte_a.parent_transaction_id 
) 


SELECT block_cat, count(distinct transaction_id) as No_of_transactions
FROM cte_b 
WHERE action = 'REQUEST' AND or_action = 'DENY_REQUEST'  
GROUP BY 1 
ORDER BY 2 DESC 

 * sqlite:///LeanTaaSTestDB.db
Done.


block_cat,No_of_transactions
3-5 hours,63
1-2 hours,55
2-3 hours,42
9 hours,5
6-8 hours,5
11 hours,4


In [266]:
%%sql 

with cte as ( 
    SELECT *, 
        round((JULIANDAY(end_time)-JULIANDAY(start_time))*24,1) as block_duration
    FROM exchange_transactions
),
cte_a as (
    SELECT *, 
    case 
        when block_duration >=1 and block_duration <2 then '1-2 hours' 
        when block_duration >=2 and block_duration <3  then '2-3 hours' 
        when block_duration >=3 and block_duration <6 then '3-5 hours' 
        when block_duration >=6 and block_duration <=8 then '6-8 hours' 
        when block_duration >=9 and block_duration <10 then '9 hours'
        when block_duration >=10 and block_duration <11 then '10 hours'
        when block_duration >=11 and block_duration <12 then '11 hours'
        when block_duration >=12 then '12 hours'
        else 0
    end as block_cat  
    FROM cte
),
cte_b as(
    select at.*, cte_a.block_duration, cte_a.block_cat, 
        cte_a.scheduler as or_scheduler, cte_a.action as or_action, 
        cte_a.created_datetime 
    from exchange_transactions at join cte_a on at.transaction_id = cte_a.parent_transaction_id 
), cte_c as(
SELECT location, block_cat, 
    avg(block_duration) as avg_hrs, 
    count(distinct transaction_id) as count
FROM cte_b 
WHERE action = 'REQUEST' 
    AND or_action = 'DENY_REQUEST'  
GROUP BY 1,2 
)
SELECT location, block_cat, round(avg_hrs,2), count 
FROM (
    SELECT  location, block_cat, count, avg_hrs , 
        rank() over (partition by location order by count desc)as rank 
    FROM cte_c  
)
WHERE rank = 1 
ORDER BY count DESC

 * sqlite:///LeanTaaSTestDB.db
Done.


location,block_cat,"round(avg_hrs,2)",count
MRAH OR,1-2 hours,1.3,32
CENTER,3-5 hours,4.33,15
MASC OR,3-5 hours,3.94,9
MSC OR,2-3 hours,2.29,7
VISION OR,1-2 hours,1.07,7
MRH OR,3-5 hours,4.7,5
CENTER PEDS,3-5 hours,4.0,2


In general, blocks with short duration are more likely to have their OR reqeusts denied in comparison to blocks with long duration. 
In MRAH, most of the block request denied lasted had a duration of 1-2 hours. In Center, most of the block request denied lasted had a duration of 3-5 hours.

We have a Tableau viz for the table above (Avg block duration by location)

#### Narration of the chart:

- All locations are consistent in having blocks of short duration being denied the most, where short is either from 1-2 hours, or from 3-5 hours.

Note: there is a slight discrepeny between the average shown in the chart versus that in the table above.

# Result and Summary

Center has the highst adoption rate, measured by percentage of request approved - 90%. The next is MRH with 87% approval rate. 

On average, Central PEDS has the highest request denial rate at 22%. The second highest is MSC OR with 17%, followed by MRAH OR and VISION OR, both of which has a 15% request denial rate. Center has the lowest denial rate at 6%.
On average, Vision has the highest transfer denial rate at 10%, followed by Center and MSC OR with 1% denial rate. 

Blocks with short durations, i.e. 1-2 hours, 3-5 hours, faced the most request denial. However, our dataset is insufficient for us to identify the "Why" of this pattern.

# Improvements and next steps

#### We can certainly look at other cohorts to identify problematic areas in driving adoption:
- How does the start time of a block relate to request denital or transfer denial?
- What is the average time it takes for a request, a approve_transfer, or a approve_request to be marked Mark_Update? Any abnormalities in this can reveal potention issues  or failures with the platform. 
- Which surgeon had the most request denial? transfer denial? And why is that? 
- Is there any association between the time gap from transaction created date to snapshot date and request/transfer denial?  

#### Other questions: 
- What is the customer's primary area of concern for identifying new opportunities? Which of the cohorts, mentioned or not alreay mentioned, would be valuable to investigate?  
- What's special about VISION, CENTER, and MSC OR that lead to their transfer getting denied? Or would it be too early for us to point these three locations as problematic area given the limited data we have? 



# Limitations


1.  It's crucial to understand the gap between the range of created date amd the range of snapshot date. Why were there no adjustments made to block schedules from January to May? Was this the intended usage of the Exchange platform? Just to target three months of the year?  
    The reason for this phenomeon could support or overturn any month-over-month behavior or trend-related oberservations we've made so far.  
    
 
2. Our data contains transactions created from January to August only. A dataset of such a small size limits our abilities to capture potential hidden factors, seasonality,  or other seasonal-effects throughout the year, all of which could have an impact on the adoption rate of the new launch.